# Import whatever packages you need here

In [1]:
import requests



# This was the code you wrote

In [2]:
# args for GET request
url = 'my_url'
user = 'my@email.com' + '/token'
pwd = 'my_secret_passwrod'

response = requests.get(url, auth=(user, pwd))

if response.status_code != 200:
    print('Status: ', response.status_code)

# serials
data = response.json()


# Explore the data

In [13]:
# View the whole data blob
# display(data)  # commented out because it takes too much screen space

print('See what type the data is')
print(type(data))

print()
print('Its a dict, so look at the keys')
print(data.keys())

print()
print('Check out what type the "tickets" value is since thats probably what we want')
print(type(data['tickets']))

print()
print('okay, its a list, indexed by integers, lets look at the tupe of the first item in the list')
print(type(data['tickets'][0]))


print()
print('okay, its a dict, lets look at its keys')
print(data['tickets'][0].keys())

See what type the data is
<class 'dict'>

Its a dict, so look at the keys
dict_keys(['tickets', 'count', 'end_of_stream', 'next_page', 'end_time'])

Check out what type the "tickets" value is since thats probably what we want
<class 'list'>

okay, its a list, indexed by integers, lets look at the tupe of the first item in the list
<class 'dict'>

okay, its a dict, lets look at its keys
dict_keys(['url', 'id', 'external_id', 'via', 'created_at', 'updated_at', 'type', 'subject', 'raw_subject', 'description', 'priority', 'status', 'recipient', 'requester_id', 'submitter_id', 'assignee_id', 'organization_id', 'group_id', 'collaborator_ids', 'follower_ids', 'email_cc_ids', 'forum_topic_id', 'problem_id', 'has_incidents', 'is_public', 'due_at', 'tags', 'custom_fields', 'satisfaction_rating', 'sharing_agreement_ids', 'fields', 'followup_ids', 'ticket_form_id', 'brand_id', 'satisfaction_probability', 'allow_channelback', 'allow_attachments', 'generated_timestamp'])


# Let's make a list of records we want to read

In [16]:
tickets = data['tickets']

# Let's write a function that knows how to pull info out of one ticket

In [21]:
def extract(ticket):
    """
    This function extracts record dictionary I care about from a single ticket dictionary
    """
    # Create an empty record that will hold our output
    record = {}
    
    # If the ticket contains a "via" field, get it otherwise get an empty dict
    via = ticket.get('via', {})
    
    # Same rule for source, etc.
    source = via.get('source', {})
    source_from = source.get('from', {})
    
    # Here we do something different.  If I can't find address,
    # say the address was "None"
    address = source_from.get('address', None)
    record['address'] = address
    
    # The above was for grabbing nested stuff.  Below is the simple top-level stuff
    
    # Define the top level fields you want to grab
    fields = ['satisfaction_probability', 'assignee_id']    
    
    # Loop over those fields
    for field in fields:
        # Assign record from the corresponding ticket field
        record[field] = ticket[field]
    
    return record

# Apply this function to all members of our list of tickets

In [33]:
# Transform tickets to records with our function using a "list comprehension"
records = [extract(ticket) for ticket in tickets]

# Show the first three records
rr = list(records[:5])
rr

[{'address': 'kevin.kapostasy@igs.com',
  'satisfaction_probability': 0.33669,
  'assignee_id': 380454819451},
 {'address': 'jdhankhar@cvent.com',
  'satisfaction_probability': 0.3088,
  'assignee_id': 380454819451},
 {'address': 'jdhankhar@cvent.com',
  'satisfaction_probability': 0.09761,
  'assignee_id': 389367924132},
 {'address': 'amarin@bitdefender.com',
  'satisfaction_probability': 0.153797,
  'assignee_id': 380454819451},
 {'address': 'mindy.wittman@ambition.com',
  'satisfaction_probability': 0.346716,
  'assignee_id': 389367924132}]

# This is how I would look stuff up

In [37]:
# Create a dictionary that maps, say ID numbers to something you care about
lookup = {
    380454819451: 'ryan',
    389367924132: 'brady'
}

# Loop over all records
rr = list(records[:5])
for rec in rr:
    # Make a new field with a name corresponding to what you care about
    # Fill it with the lookup based on the data you have
    rec['name'] = lookup[rec['assignee_id']]
    print(rec)

{'address': 'kevin.kapostasy@igs.com', 'satisfaction_probability': 0.33669, 'assignee_id': 380454819451, 'name': 'ryan'}
{'address': 'jdhankhar@cvent.com', 'satisfaction_probability': 0.3088, 'assignee_id': 380454819451, 'name': 'ryan'}
{'address': 'jdhankhar@cvent.com', 'satisfaction_probability': 0.09761, 'assignee_id': 389367924132, 'name': 'brady'}
{'address': 'amarin@bitdefender.com', 'satisfaction_probability': 0.153797, 'assignee_id': 380454819451, 'name': 'ryan'}
{'address': 'mindy.wittman@ambition.com', 'satisfaction_probability': 0.346716, 'assignee_id': 389367924132, 'name': 'brady'}


# Now Pretend I didn't have a lookup for everyone

In [39]:
# Create a dictionary that maps, say ID numbers to something you care about
lookup = {
    389367924132: 'brady'
}

# Loop over all records
rr = list(records[:5])
for rec in rr:
    # Make a new field with a name corresponding to what you care about
    # Fill it with the lookup based on the data you have
    rec['name'] = lookup.get(rec['assignee_id'], 'I_dunno')
    print(rec)

{'address': 'kevin.kapostasy@igs.com', 'satisfaction_probability': 0.33669, 'assignee_id': 380454819451, 'name': 'I dunno'}
{'address': 'jdhankhar@cvent.com', 'satisfaction_probability': 0.3088, 'assignee_id': 380454819451, 'name': 'I dunno'}
{'address': 'jdhankhar@cvent.com', 'satisfaction_probability': 0.09761, 'assignee_id': 389367924132, 'name': 'brady'}
{'address': 'amarin@bitdefender.com', 'satisfaction_probability': 0.153797, 'assignee_id': 380454819451, 'name': 'I dunno'}
{'address': 'mindy.wittman@ambition.com', 'satisfaction_probability': 0.346716, 'assignee_id': 389367924132, 'name': 'brady'}
